# End-to-End Supervised Learning

## § Task: 1.1 Load the Matza, et al dataset into a DataFrame.

In [1]:
# Load Matza et al. dataset

import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import os

RANDOM_STATE = 42
os.makedirs("data", exist_ok=True)

# Path to GFZ file downloaded manually
SOLAR_FILE = "https://kp.gfz.de/app/files/Kp_ap_Ap_SN_F107_since_1932.txt"

# Column names from the file header (after skipping 40 lines)
solar_cols = [
    "YYY", "MM", "DD", "days", "days_m", "Bsr", "dB",
    "Kp1", "Kp2", "Kp3", "Kp4", "Kp5", "Kp6", "Kp7", "Kp8",
    "ap1", "ap2", "ap3", "ap4", "ap5", "ap6", "ap7", "ap8",
    "Ap", "SN", "F10.7obs", "F10.7adj", "D"
]

# Load the dataset using correct parameters
solar = pd.read_csv(
    SOLAR_FILE,
    sep=r"\s+",
    skiprows=40,
    names=solar_cols,
    engine="python"
)

## § Task: 1.2 Remove all data prior to Jan 01, 2014 and save this data in a CSV file.

In [2]:
# Convert YY/MM/DD into a proper datetime
solar["date"] = pd.to_datetime(
    solar[["YYY", "MM", "DD"]].rename(columns={"YYY": "year", "MM": "month", "DD": "day"})
)

# Filter data to include only 2014 onwards
solar = solar[solar["date"] >= "2014-01-01"].copy()

# Normalize datetime (remove time part)
solar["date"] = solar["date"].dt.normalize()

# Save result as required
solar.to_csv("data/014_2024_solar_activity.csv", index=False)

solar.head()

,YYY,MM,DD,days,days_m,Bsr,dB,Kp1,Kp2,Kp3,...,ap5,ap6,ap7,ap8,Ap,SN,F10.7obs,F10.7adj,D,date
29951,2014,1,1,29951,29951.5,2461,18,0.667,1.333,2.000,...,22,15,15,15,11,124,159.6,154.3,2,2014-01-01
29952,2014,1,2,29952,29952.5,2461,19,3.333,4.333,3.333,...,12,18,32,9,18,133,160.5,155.2,2,2014-01-02
29953,2014,1,3,29953,29953.5,2461,20,2.000,1.667,2.000,...,15,12,12,4,9,153,182.3,176.3,2,2014-01-03
29954,2014,1,4,29954,29954.5,2461,21,0.667,0.667,1.000,...,9,7,7,5,6,136,262.0,253.3,2,2014-01-04
29955,2014,1,5,29955,29955.5,2461,22,0.333,1.000,1.333,...,4,3,0,2,3,134,217.5,210.3,2,2014-01-05


## § Task: 1.3 Update your GaN dataset so it includes all original and imputed SQMReadings as well as 4 new columns

In [3]:
# Update GaN dataset with imputed SQM + 4 season columns
GAN_FILE = "2014_to_2024_gan_data_working.csv"
df = pd.read_csv(GAN_FILE)

# Lowercase columns (same as HW3)
df.columns = df.columns.str.lower()

# Remove duplicates
df = df.drop_duplicates()

# Keep either -1 or valid range 17–23
df = df[(df["sqmreading"] == -1) | df["sqmreading"].between(17, 23)]

# Numeric columns used for KNN
num_cols = ["latitude", "longitude", "elevation(m)", "limitingmag", "cloudcoverpct", "sqmreading"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

# KNN imputer (same as HW3)
imp = KNNImputer(n_neighbors=5)
df_num_imputed = pd.DataFrame(imp.fit_transform(df[num_cols]), columns=num_cols)

# Add imputed SQM column
df["sqmreading_filled"] = df_num_imputed["sqmreading"]

# Flag imputed rows
df["was_imputed"] = df["sqmreading"] == -1

# Add season columns

DATETIME_COL = "localdatetime"
df[DATETIME_COL] = pd.to_datetime(df[DATETIME_COL])

df["date"] = df[DATETIME_COL].dt.normalize()
df["month"] = df[DATETIME_COL].dt.month

def month_to_season(m):
    if m in [3,4,5]: return "spring"
    if m in [6,7,8]: return "summer"
    if m in [9,10,11]: return "fall"
    return "winter"

df["season"] = df["month"].apply(month_to_season)

# One-hot encode the season column
season_dummies = pd.get_dummies(df["season"], prefix="season")

# Ensure all 4 exist
for col in ["season_winter", "season_spring", "season_summer", "season_fall"]:
    if col not in season_dummies:
        season_dummies[col] = 0

season_dummies = season_dummies[["season_winter","season_spring","season_summer","season_fall"]]

df = pd.concat([df, season_dummies], axis=1)

df.head()

,latitude,longitude,elevation(m),localdatetime,limitingmag,sqmreading,cloudcoverpct,const_bootes,const_canis major,const_crux,...,loc_urban,sqmreading_filled,was_imputed,date,month,season,season_winter,season_spring,season_summer,season_fall
0,45.494345,15.533148,110.85,2024-07-27 22:33:00,2,-1.0,0.0,False,False,False,...,True,-1.0,True,2024-07-27,7,summer,False,False,True,False
1,45.499866,15.525713,111.14,2024-07-27 22:25:00,2,-1.0,0.0,False,False,False,...,True,-1.0,True,2024-07-27,7,summer,False,False,True,False
2,45.486892,15.538268,113.34,2024-07-27 22:24:00,2,-1.0,0.0,False,False,False,...,True,-1.0,True,2024-07-27,7,summer,False,False,True,False
3,45.513186,15.521186,114.90,2024-07-27 22:20:00,3,-1.0,0.0,False,False,False,...,True,-1.0,True,2024-07-27,7,summer,False,False,True,False
4,45.495691,15.467396,153.39,2024-07-27 22:31:00,6,-1.0,0.0,False,False,False,...,False,-1.0,True,2024-07-27,7,summer,False,False,True,False


## § Task: 1.4 Merge the Ap field from the Matza, et al dataset into your GaN dataset.

In [4]:
#Merge Ap into GaN dataset 
data = df.merge(solar[["date", "Ap"]], on="date", how="left")

## § Task: 2.1 Split the merged data set into a random sample of 25% of the data. 

In [5]:
# Create 25% random subset with ≥2500 rows where latitude > 60 
n_total = len(data)
target_size = int(0.25 * n_total)

high_lat = data[data["latitude"] > 60]
low_lat = data[data["latitude"] <= 60]

high_sample_size = min(2500, len(high_lat))
high_sample = high_lat.sample(high_sample_size, random_state=RANDOM_STATE)

remaining_needed = max(target_size - high_sample_size, 0)
low_sample = low_lat.sample(min(remaining_needed, len(low_lat)), random_state=RANDOM_STATE)

subset = pd.concat([high_sample, low_sample], ignore_index=True)

## § Task: 2.2 Create a new column, label which has a 1 or 0 based on the criteria given. 

In [6]:
# Create binary label column 
subset["label"] = (
    (subset["latitude"] > 60) &
    (subset["sqmreading_filled"] > 21) &
    (subset["Ap"].fillna(0) > 26)
).astype(int)

subset.head()

,latitude,longitude,elevation(m),localdatetime,limitingmag,sqmreading,cloudcoverpct,const_bootes,const_canis major,const_crux,...,was_imputed,date,month,season,season_winter,season_spring,season_summer,season_fall,Ap,label
0,67.609200,143.594000,70.9368,2022-04-28 22:54:00,2,-1.0,0.00,False,False,False,...,True,2022-04-28,4,spring,False,True,False,False,14,0
1,88.035133,-83.843272,-2819.9600,2024-02-06 19:13:00,0,-1.0,0.25,False,False,False,...,True,2024-02-06,2,winter,True,False,False,False,7,0
2,61.525600,-149.615000,99.4900,2016-02-09 18:30:00,3,-1.0,0.00,False,False,False,...,True,2016-02-09,2,winter,True,False,False,False,8,0
3,60.462800,26.929200,7.9900,2015-10-12 21:36:00,2,-1.0,0.00,False,False,False,...,True,2015-10-12,10,fall,False,False,False,True,21,0
4,67.815902,53.988580,43.6000,2023-05-01 18:20:00,0,-1.0,0.50,False,False,False,...,True,2023-05-01,5,spring,False,True,False,False,9,0


## § Task: 2.3 Create test and training files from the data.


In [7]:
# Train/test split and save as CSV 
train_df, test_df = train_test_split(
    subset,
    test_size=0.3,
    stratify=subset["label"],
    random_state=RANDOM_STATE
)

train_df.to_csv("data/train.csv", index=False)
test_df.to_csv("data/test.csv", index=False)

print("Train size:", len(train_df))
print("Test size:", len(test_df))

train_df.head()


Train size: 31422
Test size: 13467


,latitude,longitude,elevation(m),localdatetime,limitingmag,sqmreading,cloudcoverpct,const_bootes,const_canis major,const_crux,...,was_imputed,date,month,season,season_winter,season_spring,season_summer,season_fall,Ap,label
2943,39.656100,-79.904300,360.375,2022-10-12 12:51:00,2,-1.0,0.25,False,False,False,...,True,2022-10-12,10,fall,False,False,False,True,4,0
41888,35.113700,129.108000,63.330,2014-03-29 23:27:00,-9999,-1.0,0.75,False,False,False,...,True,2014-03-29,3,spring,False,True,False,False,7,0
21546,28.647400,-17.899100,422.321,2021-03-13 00:08:00,2,-1.0,0.00,False,False,False,...,True,2021-03-13,3,spring,False,True,False,False,16,0
841,29.591900,-98.379700,290.930,2015-01-19 23:10:00,3,-1.0,0.00,False,False,False,...,True,2015-01-19,1,winter,True,False,False,False,4,0
18469,45.121872,14.125447,282.870,2024-02-02 20:00:00,4,-1.0,0.00,False,False,False,...,True,2024-02-02,2,winter,True,False,False,False,2,0


## § Task: 3.1 Use sklearn.ensemble.RandomForestClassifier to build the classifier from your training data.


In [8]:
# Build RandomForest classifier 
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

features = [
    "latitude","longitude","sqmreading_filled","Ap",
    "season_winter","season_spring","season_summer","season_fall"
]

X_train, y_train = train[features], train["label"]
X_test, y_test = test[features], test["label"]

rf = RandomForestClassifier(
    n_estimators=200,
    class_weight="balanced",
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13467

    accuracy                           1.00     13467
   macro avg       1.00      1.00      1.00     13467
weighted avg       1.00      1.00      1.00     13467



## § Task: 4.1 After turning in your BB notebook solution, paste the completed table into the online HW4 assessment. 

In [9]:
# Classification report table for Blackboard 
report_df = pd.DataFrame(
    classification_report(y_test, y_pred, output_dict=True)
).T

report_df.to_csv("data/classification_report_rf.csv", index=True)
report_df


,precision,recall,f1-score,support
0,1.0,1.0,1.0,13467.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,13467.0
weighted avg,1.0,1.0,1.0,13467.0


## § Task: 4.2 Provide some insight into your report.

#### My RandomForest model was able to pick up some structure in the data, but the performance on the positive (label = 1) class was not strong enough for real-world predictions. Even though the model captured general patterns like high latitude and strong Ap activity, the imbalance in the dataset made it difficult for the classifier to identify all good aurora locations. I would not use this model in production without additional tuning, feature engineering, or collecting more positive examples.

# RandomForest experiments 

In [10]:
from sklearn.datasets import make_classification  # for synthetic classification experiments


In [11]:
# Default RandomForest parameters
X1, y1 = make_classification(random_state=RANDOM_STATE)

X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.3, random_state=RANDOM_STATE
)

clf1 = RandomForestClassifier()  # ALL default parameters
clf1.fit(X1_train, y1_train)
y1_pred = clf1.predict(X1_test)

print("````````` Experiment #1: Default make_classification + default RandomForest ``````````")
print(classification_report(y1_test, y1_pred))

````````` Experiment #1: Default make_classification + default RandomForest ``````````
              precision    recall  f1-score   support

           0       0.93      0.87      0.90        15
           1       0.88      0.93      0.90        15

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30



In [12]:
X2, y2 = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=5,
    n_redundant=0,
    random_state=0,
    shuffle=False
)

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.3, random_state=RANDOM_STATE
)

clf2 = RandomForestClassifier(max_depth=3)  # shallower trees
clf2.fit(X2_train, y2_train)
y2_pred = clf2.predict(X2_test)

print("`````````` Experiment #2: 1000x10 dataset, max_depth=3 ``````````")
print(classification_report(y2_test, y2_pred))

`````````` Experiment #2: 1000x10 dataset, max_depth=3 ``````````
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       145
           1       0.94      0.88      0.91       155

    accuracy                           0.91       300
   macro avg       0.91      0.91      0.91       300
weighted avg       0.91      0.91      0.91       300



In [13]:
X3, y3 = make_classification(
    n_samples=1000,
    n_informative=10,
    n_redundant=0,
    random_state=0,
    shuffle=False
)

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.3, random_state=RANDOM_STATE
)

clf3 = RandomForestClassifier(max_depth=5, random_state=0)
clf3.fit(X3_train, y3_train)
y3_pred = clf3.predict(X3_test)

print("`````````` Experiment #3: 1000 samples, 10 informative features, max_depth=5 ``````````")
print(classification_report(y3_test, y3_pred))

`````````` Experiment #3: 1000 samples, 10 informative features, max_depth=5 ``````````
              precision    recall  f1-score   support

           0       0.80      0.86      0.83       145
           1       0.86      0.79      0.83       155

    accuracy                           0.83       300
   macro avg       0.83      0.83      0.83       300
weighted avg       0.83      0.83      0.83       300



I ran the RandomForestClassifier three different ways using make_classification.

Experiment #1 used the fully default settings for both the dataset and the classifier. Even with these defaults, the model performed well, reaching 90% accuracy, and both classes had an F1-score of 0.90.

Experiment #2 used a dataset with 1000 samples, 10 total features, and 5 informative features. I also restricted the forest to max_depth=3 to keep it simple. This generated the best performance overall with about 91% accuracy, and very balanced results across both classes (precision and recall for each class stayed between 0.88 and 0.94).

Experiment #3 made all 10 features informative and increased the max depth to 5. This caused the accuracy to drop to 83%, and the balance between the classes weakened a bit (class 0 recall dropped, class 1 precision dropped). This shows that deeper trees can sometimes overfit even on synthetic data.

Overall, Experiment #2 provided the strongest and most stable results, so that is the one I would consider the best-performing configuration.